## Filtering

The estimate the position of the Thymio, we have chosen to use an Extended Kalman Filter (EKF) due to the non-linearity of the system. Another possible option was a Particle Filter but it is less computationally efficient and, given the need for real-time estimation, the EKF was a better choice. 

The odometry and camera measurements are assumed to have Gaussian noise which aligns with the assumptions of the EKF. 

## Space state estimation 

In the EKF, the current state estimate has the following shape: 

$$ 
state_{k}= A_{k-1} \cdot state_{k-1} + B_{k-1}\cdot u_{k-1} + w_{k-1}
$$

We used a 5-states estimation : <span style="color: red;">We added the speed and the angular velocity because quoi ? il m'a dit pour l'odomeetry quand on cache la camera mais est ce que c'est vraiment ca qui l'inclu et pas juste le measurement? </span>
: $ state_{k}=\begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix}$


We can estimate the Tymio's current state from the previous one using the dynamic of the model: 
$$
\begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix} = \begin{bmatrix} 
x_{k-1} + \cos(\gamma_{k-1}) \cdot T_s \cdot v_{k-1}\\
y_{k-1} + \sin(\gamma_{k-1}) \cdot T_s \cdot v_{k-1}\\
\gamma_{k-1} + Ts \cdot \omega_{k-1}\\
v_{k-1}\\
\omega_{k-1}
\end{bmatrix} 
$$

We can rewrite the equation as : 

$$
 \begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix} = \begin{bmatrix} 
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{bmatrix} \cdot \begin{bmatrix} 
x_{k-1} \\
y_{k-1} \\
\gamma_{k-1} \\
v_{k-1}\\
\omega_{k-1}
\end{bmatrix} + \begin{bmatrix}
\cos(\gamma_{k-1}) \cdot T_s & 0 \\
\sin(\gamma_{k-1}) \cdot T_s & 0 \\
0 & T_s \\
1 & 0 \\
0 & 1
\end{bmatrix} \cdot \begin{bmatrix}
v_{k-1}\\
\omega_{k-1}
\end{bmatrix} + \begin{bmatrix} 
noise_{k-1} \\
noise_{k-1} \\
noise_{k-1} \\
noise_{k-1} \\
noise_{k-1} \\
\end{bmatrix}
$$


Hence we obtain : $ A_{k-1}=\begin{bmatrix} 
    1 & 0 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0\\
    0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 
    \end{bmatrix} \text{and } B_{k-1} = \begin{bmatrix}
\cos(\gamma_{k-1}) \cdot T_s & 0 \\
\sin(\gamma_{k-1}) \cdot T_s & 0 \\
0 & T_s \\
1 & 0 \\
0 & 1
\end{bmatrix}$


## Predicted covariance 
The predicted covariance is given by :
$$
P_{pred} =  G\cdot P_{k-1}\cdot G^T + Q
$$

The matrix $G$ is given by the Jaccobian of $A_{k-1} \cdot x_{k-1} + B_{k-1}\cdot u_{k-1}$:

$$
G = \begin{bmatrix}
1 & 0 & -\sin(\gamma_{k-1}) \cdot T_s \cdot v_{k-1} & \cos(\gamma_{k-1}) \cdot T_s & 0 \\
0 & 1 & \cos(\gamma_{k-1}) \cdot T_s \cdot v &_{k-1} \sin(\gamma_{k-1}) \cdot T_s & 0 \\
0 & 0 & 1 & 0 & T_s \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
$$



## Covariance matrices
**State covariance** :
$ Q = \begin{bmatrix}
q_x & 0 & 0 & 0 & 0 \\
0 & q_y & 0 & 0 & 0 \\
0 & 0 & q_{\gamma} & 0 & 0 \\
0 & 0 & 0 & q_v & 0 \\
0 & 0 & 0 & 0 & q_{\omega}
\end{bmatrix}$

**Measurement covariance** : 
- Camera available : $R_{\text{camera}} = \begin{bmatrix}
r_x & 0 & 0 & 0 & 0 \\
0 & r_y & 0 & 0 & 0 \\
0 & 0 & r_{\gamma} & 0 & 0 \\
0 & 0 & 0 & r_v & 0 \\
0 & 0 & 0 & 0 & r_{\omega}
\end{bmatrix}$

- Camera not available : $R_{\text{noCamera}} = \begin{bmatrix}
r_v & 0 \\
0 & r_{\omega}
\end{bmatrix}
$

**Choice of the parameters:**

<span style="color: red;">You can find all the calculations in covariance_estimation_whith_turn.ipynb<span>

We determined the parameters via a little experiment: we gave the instruction to the thymio to go straight to a certain speed (50 thymio speed for both wheel) and measured the actual speed of the wheels.

The avergage speed is calculated as : $avg\_ speed =\frac{v_{left}+v_{right}}{2}$

![Alt Text](images/speeds_exp-cov_go_straight.png)


The measured distance that the robot did is: d=286mm.

Hence, we can calculate a conversion factor from Thymio-speed to mm/s : 
$$
thymio\_ speed\_ to\_ mms= \frac{d}{Ts*nb_{timeSteps}*thymio\_ speed}=0.430$$

**Covariances of v and $\omega$**

To have the most accurate covariances as possible, we reccorded the thymio wheels speed on a curve trajectory. To to that we used (motor_left,motor_right)=(45,55).

For all t : 
- $avg\_ speed (t) =\frac{v_{left}(t)+v_{right}(t)}{2}$
- $omega (t) =\frac{v_{left}(t)-v_{right}(t)}{L}$, with L=distance between the 2 wheels of the robot.

Assuming that half of variance is caused by perturbation of state and  half by measurement, we obtain :
- $q_v=\frac{var(avg\_ speed)}{2}=2.29 [mm^2/s^2]$
- $r_v=\frac{var(avg\_ speed)}{2}=2.29 [mm^2/s^2]$

- $q_w=\frac{var(omega)}{2}=0.0008 [rad^2/s^2]$
- $r_w=\frac{var(omega)}{2}=0.0008 [rad^2/s^2]$

**Covariances of x, y and $\gamma$**
- $q_x=var(x_{measured}-x_{expected})=0.78 [mm^2/s^2]$
- $q_y=var(y_{measured}-y_{expected})=13.39 [mm^2/s^2]$
- $q_\gamma=var(\gamma_{measured}-\gamma_{expected})=0.00016 [rad^2/s^2]$


## Observation/measurement model 

The observation modelis given by : 
$$
y_k = H \cdot x_k + \nu_k 
$$

- Camera available:
$$
\begin{bmatrix} 
y_1^k \\
y_2^k \\
y_3^k \\
y_4^k \\
y_5^k
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix} \cdot \begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix} + \begin{bmatrix} 
noise_{k-1}^1 \\
noise_{k-1}^2 \\
noise_{k-1}^3 \\
noise_{k-1}^4 \\
noise_{k-1}^5 \\
\end{bmatrix}
$$

- Camera not available:
$$
\begin{bmatrix} 
y_4^k \\
y_5^k
\end{bmatrix} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix} \cdot \begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix} + \begin{bmatrix} 
noise_{k-1}^4 \\
noise_{k-1}^5
\end{bmatrix}
$$

The observations are given by the camera for $x_{k}$, $y_{k}$ and $\gamma_{k}$ and by the odometry for $v_{k}$ and $\omega_{k}$.

## Sources 
- Course "Basics of Mobile Robotics" by Francesco Mondada
- Automatic Addison https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
- Wikipedia page of the Extended Kalman Filter : https://en.wikipedia.org/wiki/Extended_Kalman_filter#Discrete-time_predict_and_update_equations
- Chat GPT